# Grid4x4 - PettingZoo + RLlib

In [1]:
import os

import ray
from ray.tune.registry import register_env

In [2]:
import random
import numpy as np
import torch

SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Environment Setup

In [3]:
import supersuit as ss
from reward_functions import combined_reward

env_name = "grid2x2"

## Training the RL Agent

In [4]:
from envs import CountAllRewardsEnv
from observation import Grid2x2ObservationFunction

env_params = {
    "net_file": os.path.join("nets","grid4x4","grid4x4.net.xml"),
    "route_file": os.path.join("nets","grid4x4","grid4x4_1.rou.xml"),
    "num_seconds": 3600,
    "reward_fn": combined_reward,
    "sumo_seed": SEED,
    "observation_class": Grid2x2ObservationFunction
}
env = CountAllRewardsEnv(**env_params)
env = ss.frame_stack_v1(env, 3)

KeyError: 'A0'

In [5]:
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv

ray.init()

register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

2023-05-29 15:42:45,258	INFO worker.py:1625 -- Started a local Ray instance.


In [6]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

config = (
    PPOConfig()
    .environment(env=env_name)
    .framework(framework="torch")
    .rollouts(
        num_rollout_workers=4, rollout_fragment_length=128,
        # num_envs_per_worker=5,
        # batch_mode="truncate_episodes",
    )
    .training(
        train_batch_size=512,
        lr=2e-5,
        gamma=0.99,
        lambda_=0.9,
        use_gae=True,
        clip_param=0.4,
        grad_clip=None,
        entropy_coeff=0.1,
        vf_loss_coeff=0.25,
        sgd_minibatch_size=64,
        num_sgd_iter=10,
        # lambda_=0.95,
        # kl_coeff=0.5,
        # clip_param=0.1,
        # vf_clip_param=10.0,
        # entropy_coeff=0.01,
        # train_batch_size=5000,
        # sgd_minibatch_size=500,
        # num_sgd_iter=10,
    )
    # .evaluation(
    #     evaluation_duration=3600,
    #     evaluation_duration_unit="timesteps",
    #     evaluation_num_workers=1,
    # )
    .debugging(log_level="WARN", seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)

In [7]:
from time import time
from ray import tune

current_time = int(time())
dir_path = os.path.join("ray_results", env_name, str(current_time))

tune.run(
    "PPO",
    name=f"PPO_{str(current_time)}",
    stop={"timesteps_total": 2e5},
    checkpoint_freq=10,
    local_dir=dir_path,
    config=config.to_dict(),
)

c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\tune\experiment\experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(PPO pid=15072) 2023-05-29 15:42:54,646	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=15072) 2023-05-29 15:42:55,116	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=7016) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 59ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=19248) 2023-05-29 15:43:03,369	WARNING env.py:285 -- Your MultiAgentEnv <ParallelPettingZooEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(RolloutWorker pid=19248) 2023-05-29 15:43:55,783	WARNING env_runner_v2.py:154 -- More than 2048 observations in 128 env steps for episode 330665021975472663 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_grid4x4_1593c_00000,1671168,"{'ObsPreprocessorConnector_ms': 0.015610933303833008, 'StateBufferConnector_ms': 0.011094808578491211, 'ViewRequirementAgentConnector_ms': 0.7719197273254395}","{'num_env_steps_sampled': 104448, 'num_env_steps_trained': 104448, 'num_agent_steps_sampled': 1671168, 'num_agent_steps_trained': 1671168}",{},2023-05-29_18-35-33,False,720,{},-154707,-157111,-159724,0,144,JM-M16,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.5467941053502727, 'cur_kl_coeff': 0.22500000000000003, 'cur_lr': 2e-05, 'total_loss': 2.271271016728133, 'policy_loss': -0.00677550317850546, 'vf_loss': 9.911993718147277, 'vf_explained_var': -5.774199962615967e-09, 'kl': 0.014711259533578413, 'entropy': 2.0326193901710212, 'entropy_coeff': 0.09999999999999999}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 64.0, 'num_grad_updates_lifetime': 260480.5, 'diff_num_grad_updates_vs_sampler_policy': 639.5}}, 'num_env_steps_sampled': 104448, 'num_env_steps_trained': 104448, 'num_agent_steps_sampled': 1671168, 'num_agent_steps_trained': 1671168}",204,127.0.0.1,1671168,1671168,104448,512,104448,512,0,4,0,0,512,"{'cpu_util_percent': 4.682812500000001, 'ram_util_percent': 38.501562500000006, 'gpu_util_percent0': 0.0, 'vram_util_percent0': 0.0}",15072,{},{},{},"{'mean_raw_obs_processing_ms': 3.3925715709297357, 'mean_inference_ms': 1.2238536786896537, 'mean_action_processing_ms': 0.7824720522790197, 'mean_env_wait_ms': 326.81172288139265, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -154706.97841973815, 'episode_reward_min': -159723.637988752, 'episode_reward_mean': -157110.57859392036, 'episode_len_mean': 720.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-156582.14678673557, -157198.4151821003, -155266.64931842024, -157562.31699350945, -158646.59697224485, -156584.57590543566, -157906.60368303556, -157550.3169386479, -158795.29412694369, -157736.34551755476, -158284.59793015, -155022.58485800453, -158237.44848742546, -156072.7689951435, -156927.2167503601, -157328.76031556696, -158646.61330687243, -157099.0150906293, -158343.61798867316, -156116.63008901308, -158834.19776869853, -158000.38794745668, -157749.3749630917, -157967.93793685327, -155883.98208064033, -156194.13638523186, -158451.89176412555, -157225.3889675915, -155129.76026229662, -156748.02823330992, -156437.4035916598, -157636.66956022236, -156891.44181064182, -157154.41970573246, -157060.93500676707, -157221.43022781124, -157640.40654965295, -156872.9239083637, -156117.48843430847, -158447.69377881466, -157443.10882636916, -156493.37546931833, -158117.70727285303, -157065.27318233918, -158352.59157821818, -158042.73055714308, -157065.0815222455, -157013.15238599948, -157640.2658089524, -157245.06581649132, -157186.335800258, -158242.71371198285, -158406.2407901952, -156317.36413301123, -156982.02570204702, -156636.09340061317, -157337.19757264957, -157438.51259468345, -156714.5871859038, -158082.24596345902, -156673.190164252, -158049.49496476477, -155423.55627864855, -156775.13722888127, -157893.0027143028, -158174.88928725032, -156857.15939443713, -156806.5

...

(RolloutWorker pid=19248) Warning: Vehicle '1152' performs emergency braking on lane ':B1_33_1' with decel=9.00, wished=4.50, severity=1.00, time=2517.00.


Step #2500.00 (1ms ~= 1000.00*RT, ~87000.00UPS, TraCI: 223ms, vehicles TOT 1175 ACT 87 BUFTOT 1 ACT 1 BUF 0)                     
Step #2500.00 (0ms ?*RT. ?UPS, TraCI: 229ms, vehicles TOT 1175 ACT 93 BUF 0)              CI: 267ms, vehicles TOT 907 ACT 207 BU
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 224ms, vehicles TOT 1473 ACT 39 BUF 0)              OT 1 ACT 1 BUF 0)                     
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 227ms, vehicles TOT 1473 ACT 36 BUF 0)              s TOT 1220 ACT 89 BUF 0)              
Step #3600.00 (1ms ~= 1000.00*RT, ~34000.00UPS, TraCI: 225ms, vehicles TOT 1473 ACT 34 BUFCI: 240ms, vehicles TOT 1220 ACT 98 BUF
Step #3600.00 (1ms ~= 1000.00*RT, ~37000.00UPS, TraCI: 217ms, vehicles TOT 1473 ACT 37 BUFTOT 1 ACT 1 BUF 0)                     


(RolloutWorker pid=7016) Warning: Vehicle '354' performs emergency braking on lane ':A1_24_0' with decel=9.00, wished=4.50, severity=1.00, time=1370.00. [repeated 2x across cluster]


Step #1300.00 (1ms ~= 1000.00*RT, ~86000.00UPS, TraCI: 229ms, vehicles TOT 386 ACT 86 BUF OT 1 ACT 1 BUF 0)                     


2023-05-29 21:13:31,723	INFO tune.py:945 -- Total run time: 19842.15 seconds (19841.87 seconds for the tuning loop).


In [8]:
ray.shutdown()

Step #1845.00 (1ms ~= 1000.00*RT, ~197000.00UPS, TraCI: 8002ms, vehicles TOT 764 ACT 197 Bs TOT 432 ACT 93 BUF 0)               
Step #1845.00 (1ms ~= 1000.00*RT, ~191000.00UPS, TraCI: 7659ms, vehicles TOT 764 ACT 191 BTOT 1 ACT 1 BUF 0)                     
Step #1845.00 (2ms ~= 500.00*RT, ~92500.00UPS, TraCI: 8057ms, vehicles TOT 764 ACT 185 BUFOT 1 ACT 1 BUF 0)                     
Step #1845.00 (1ms ~= 1000.00*RT, ~191000.00UPS, TraCI: 8374ms, vehicles TOT 764 ACT 191 BTOT 1 ACT 1 BUF 0)                     


## Play the Trained Agent

In [39]:
checkpoint_path = os.path.join(dir_path,"PPO","PPO_grid4x4_1593c_00000_0_2023-05-29_15-42-49","checkpoint_000390")
checkpoint_path = os.path.abspath(checkpoint_path)

In [40]:
from observation import Grid4x4ObservationFunction
from helper_functions import pz_env

def env_creator():
    env_params = {
        "net_file": os.path.join("nets","grid4x4","grid4x4.net.xml"),
        "route_file": os.path.join("nets","grid4x4","grid4x4_1.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid4x4ObservationFunction
    }
    env = pz_env(**env_params)
    env = ss.frame_stack_v1(env, 3)
    return env

In [41]:
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv

env = env_creator()
register_env(env_name, lambda config: PettingZooEnv(env_creator()))

In [42]:
from ray.rllib.algorithms.ppo import PPO

ray.init()

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-05-29 22:09:02,815	INFO worker.py:1625 -- Started a local Ray instance.


(RolloutWorker pid=11256) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 52ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=26516) 2023-05-29 22:09:13,680	WARNING env.py:285 -- Your MultiAgentEnv <PettingZooEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


In [43]:
reward_sum = 0
frame_list = []
i = 0
env.reset()

In [44]:
import csv
import traci
from listeners import SimListener

csv_path = os.path.join(checkpoint_path,"play.csv")

with open(csv_path, "w", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "avg_speed",
                         "pressure", "queued", "tyre_pm", "waiting_time"])

listener = SimListener(env, csv_path, checkpoint_path)
traci.addStepListener(listener)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    
    reward_sum += reward
    if termination or truncation:
        action = None
    else:
        action = ppo_agent.compute_single_action(observation)

    env.step(action)
    i += 1

env.close()

if hasattr(listener, "tb_writer"):
    listener.tb_writer.close()

In [45]:
# Interpret results
import pandas as pd

play_csv = pd.read_csv(csv_path)
arrived = sum(play_csv.arrived_num)
tyre_pm = sum(play_csv.tyre_pm)
final_wait = play_csv.waiting_time.iat[-1]

print("Reward:", reward_sum)
print("Vehicles arrived:", arrived)
print("Total tyre PM emitted:", tyre_pm)
print("Last step waiting time:", final_wait)

-157418.18592983545


In [46]:
ray.shutdown()

(RolloutWorker pid=9276) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 209ms, vehicles TOT 0 ACT 0 BUF 0)                     [repeated 7x across cluster]


## RLlib Docs

In [ ]:
algo = config.build()

In [ ]:
# Changes made in ParallelPettingZooEnv's reset() method: see line 202-206 in
# C:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\env\wrappers\pettingzoo_env.py

algo.evaluate()

In [ ]:
from ray.tune.logger import pretty_print

result = algo.train()
print(pretty_print(result))

In [ ]:
algo.evaluate()